# Automatic email responder

This project is divided into three main phases:
1) Email retrieving
2) Generate response
3) Create response

#### Requirements

In [1]:
import os.path

# OPEN-AI API
# pip install openai
import openai
from openai import OpenAI

# GMAIL API
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

## Email retrieving

Authenticating into Gmail through Gmail API.

For this purpose, a specific google user was created: *emmanueltestpiazzatest@gmail.com* 

In [2]:
# credentials and authorizations
creds = None
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly",
          "https://www.googleapis.com/auth/gmail.modify",
          "https://www.googleapis.com/auth/gmail.compose"]

# authenticate
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)  
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    # save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

Reading incoming and unreaded messages

In [3]:
try:
    service = build("gmail", "v1", credentials=creds)
    results = service.users().messages().list(userId="me", q="{is:unread AND label:inbox}").execute()
    message_list = results.get("messages", [])

    if not message_list:
        print("No messages found.")
    else:    
        print("Incoming messages:\n")
        for message in message_list:
            result = service.users().messages().get(userId="me",id=message["id"]).execute()
            for elem in result["payload"]["headers"]:
                if(elem["name"] in ["Subject", "From"]):
                    print(elem["name"],": ", elem["value"])
            question = result["snippet"]
            print("Snippet : ", result["snippet"], "\n")

except HttpError as error:
    print(f"An error occurred: {error}")

Incoming messages:

From :  Emmanuel Piazza <emmanuelpiazza2020@gmail.com>
Subject :  Domanda
Snippet :  Ciao, questa mail è una prova per vedere se l&#39;api funziona bene 



## Generate response

Retrieve OpenAI API key

In [4]:
text_file = open("openai-key.txt", "r")
api_key = text_file.read()
text_file.close()
os.environ['OPENAI_API_KEY'] = "insert-openAI-KEY-here"

Generate prompt used for establish the context for the response message

In [77]:
prompt = "We own an online store. Manage clients feedback about our products and services. Always be kind. Shipments between 4-5 days. Return policy at 'http://onlinefurnaces.com/return'. For complains, support service number: 555-123456789"

In [78]:
question = "I bought a chair but i don't want it anymore! it's not like in the photo"

In [79]:
def generate_response(question):
    client = OpenAI()
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "assistant", "content": prompt},
        {"role": "user", "content": question}
      ]
    )
    print(response)
    return response

In [80]:
response = generate_response(question)

ChatCompletion(id='chatcmpl-8kEBJLGIwMTYdZ9piK2FAA4D4pygk', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="We're sorry to hear that you're unsatisfied with your purchase. Please refer to our return policy at 'http://onlinefurnaces.com/return' for information on how to proceed with returning the chair. If you have any further questions or require assistance, please contact our support service at 555-123456789. We apologize for any inconvenience caused and appreciate your understanding.", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1706028293, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=77, prompt_tokens=85, total_tokens=162))


In [81]:
response.choices[0].message.content

"We're sorry to hear that you're unsatisfied with your purchase. Please refer to our return policy at 'http://onlinefurnaces.com/return' for information on how to proceed with returning the chair. If you have any further questions or require assistance, please contact our support service at 555-123456789. We apologize for any inconvenience caused and appreciate your understanding."

## Create response

Retrieve only needed informations from email

In [4]:
def create_msg_obj(msg):
    
    obj = {}
    obj['threadId'] = msg['threadId']
    obj['snippet'] = msg['snippet']
    obj['id'] = msg['id']
    for elem in msg['payload']['headers']:
        if(elem["name"] in ['To', 'From', 'Subject']):
            obj[elem["name"]] = elem["value"]
            print(elem["name"],": ", elem["value"])        
    return obj

Mark incoming mail as readed in order to not generate a response more than once

In [37]:
def mark_as_readed(msg_id):
    
    try:
        service = build("gmail", "v1", credentials=creds)
        results = service.users().messages().modify(userId="me", id=msg_id, body={'removeLabelIds':["UNREAD"]}).execute()
        print("Message [",msg_id,"] labeled as readed")
        
    except Exception as e:
        print("Impossible to mark message [",msg_id,"] - e:",e)

Creating response inside Draft ('Bozze') folder.

In [5]:
from email.message import EmailMessage
import base64

def create_response(msg, reply):
    
    end = "\n\nRisposta generata automaticamente da ChatGPT."
    try:
        message = EmailMessage()
        message.set_content(reply + end)
        message["To"] = msg['To']
        message["From"] = msg['From']
        message["Subject"] = "RE:"+ msg['Subject']
        draft_obj = {
            "message":{
                'raw': base64.urlsafe_b64encode(message.as_string().encode()).decode(),
                'threadId': msg['threadId']
            }
        }
    
        service = build("gmail", "v1", credentials=creds)
        draft = service.users().drafts().create(userId="me", body=draft_obj).execute()
        if(draft):
            print("Draft message correctly created")
        
        
    except Exception as e:
        print("Impossible to create draft message - e:",e)

## TEST

In [ ]:
prompt = "We own an online store. Manage clients feedback about our products and services. Always be kind. Shipments between 4-5 days. For complains, support service number: 555-123456789"

In [42]:
try:
    service = build("gmail", "v1", credentials=creds)
    results = service.users().messages().list(userId="me", q="{is:unread AND label:inbox}").execute()
    message_list = results.get("messages", [])

    if not message_list:
        print("No new incoming messages found.")
    else:    
        print("Incoming messages:\n")
        for message in message_list:
            result = service.users().messages().get(userId="me",id=message["id"]).execute()
            msg = create_msg_obj(result)
            reply = generate_response(prompt, msg)
            reply = reply.choices[0].message.content
            mark_as_readed(msg['id'])
            create_response(msg, reply)
            print("\n\n\n")

except HttpError as error:
    print(f"An error occurred: {error}")

No new incoming messages found.
